<a href="https://colab.research.google.com/github/AhmetFarukBilgin/ML-AI_Projects/blob/main/Translate_UCI_Try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pdfplumber


In [ ]:
import os
os.listdir()
import pdfplumber

In [ ]:
#PDF → SAYFA BAZLI HAM METİN
pages_text = []

with pdfplumber.open("UCI_MTB_Rules_-_Part_IV_-_Rule_Changes_on_01.01.2026_-_ENG.pdf") as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        if text:
            pages_text.append({
                "page": i + 1,
                "text": text
            })

print("Toplam sayfa:", len(pages_text))
print("\n--- ÖRNEK SAYFA ---\n")
print(pages_text[0]["text"][:1000])

In [ ]:
#HAM METNİ BİRLEŞTİR (AMA HENÜZ PARÇALAMA YOK
raw_text = "\n".join(p["text"] for p in pages_text)

print(raw_text[:1500])


In [ ]:
#İLK TEMİZLİK (ÇOK HAFİF)
clean_lines = []

for line in raw_text.split("\n"):
    line = line.strip()
    if not line:
        continue
    if len(line) < 5:
        continue
    clean_lines.append(line)

clean_text = " ".join(clean_lines)

print(clean_text[:1500])


In [ ]:
def remove_headers_footers(text):
    text = re.sub(r"Page\s+\d+\s*/\s*\d+", "", text, flags=re.IGNORECASE)

    text = re.sub(
        r"Allée\s+Ferdi\s+Kübler\s+12.*?Switzerland",
        "",
        text,
        flags=re.IGNORECASE | re.DOTALL
    )

    text = re.sub(r"T:\s*\+?\d[\d\s]+", "", text)
    text = re.sub(r"E:\s*\S+@\S+", "", text)

    return text

In [ ]:
#Madde bazlı parçalama
import re

# SADECE "4.x.xxx" UCI maddelerini böl
parts = re.split(r"(?=\n?4\.\d+\.\d{3}\b)", clean_text)

articles = []

for part in parts:
    part = part.strip()
    if re.match(r"4\.\d+\.\d{3}\b", part):
        articles.append(part)

print("Bulunan madde sayısı:", len(articles))
print("\n--- ÖRNEK MADDE ---\n")
for article in articles[:5]:
    print(article)
    print("/n")

In [ ]:
#Maddeleri alt parçalara bölme
def split_uci_article(article_text):
    article_text = article_text.strip()

    match = re.match(r"(4\.\d+\.\d{3})(.*)", article_text, re.S)
    if not match:
        print("⚠️ Madde ID bulunamadı")
        return [article_text]

    article_id = match.group(1)
    body = match.group(2)

    # Satır kırılımlarını temizle
    body = re.sub(r"\s*\n\s*", " ", body)

    sentences = re.split(r"(?<=\.)\s+", body)

    chunks = []
    for s in sentences:
        s = s.strip()
        if re.search(r"(Allée|Switzerland|T:|E:)", s):
            continue
        if len(s) > 20:
            chunks.append(f"{article_id} {s}")

    # DEBUG ÇIKTISI
    print(f"\n📌 {article_id} için {len(chunks)} parça bulundu:")
    for c in chunks:
        print("-", c)
        print()

    return chunks


In [ ]:
split_uci_article(articles[0])


In [ ]:
#CSV Export from chunks
import csv

all_chunks = []
counters = {}

for art in articles:
    chunks = split_uci_article(art)
    for ch in chunks:
        article_id = ch.split()[0]
        text = ch[len(article_id):].strip()

        counters.setdefault(article_id, 0)
        counters[article_id] += 1

        uid = f"{article_id}_{counters[article_id]}"
        all_chunks.append([uid, article_id, text])

with open("uci_rules_en_chunks.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["uid", "article_id", "source_en"])
    writer.writerows(all_chunks)

print("✅ CSV oluşturuldu: uci_rules_en_chunks.csv")
print("Toplam satır:", len(all_chunks))


In [ ]:
import pandas as pd

INPUT_CSV = "uci_rules_en_chunks.csv"

df = pd.read_csv(INPUT_CSV, encoding="utf-8")

df.head()


In [ ]:
df.info()
df.isnull().sum()


In [ ]:
!pip install -U google-generativeai


In [ ]:
def call_teacher_llm(text):
    response = client.chat.completions.create(
        model="gemini-2.5-flash", # Güncellenmiş model adı
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content.strip()

In [ ]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('Gemini')
genai.configure(api_key=api_key)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello, just say OK")

print(response.text)


In [ ]:
test_prompt = """
You are a professional technical translator.

Translate the following text to Turkish:

Riders shall wear helmets at all times.
"""

response = model.generate_content(test_prompt)
print(response.text)


In [ ]:
SYSTEM_PROMPT = """
You are a professional technical translator specialized in UCI cycling regulations.

IMPORTANT:
You MUST strictly follow the terminology rules below.
Do NOT simplify legal language.
Do NOT replace modal verbs with future tense.
Violations are not allowed.

Your task:
- Translate the given regulatory text from English to Turkish
- Preserve ALL technical and legal terminology accurately
- Use formal regulatory Turkish (resmi yönetmelik dili)
- Do NOT simplify, summarize, or paraphrase
- Keep the sentence structure as close as possible to the original
- Maintain consistency across fragments belonging to the same article

Critical terminology rules (MANDATORY):
- The following terminology MUST be used exactly as specified
- Do NOT invent alternatives
- Respect notes and forbidden usages strictly

Terminology glossary and usage rules:

Institutional / Organizational:
- "UCI (Union Cycliste Internationale)" → "UCI (Uluslararası Bisiklet Birliği)"
  Rule: Write full form on first occurrence, then use "UCI" only
- "National Federation" → "Ulusal Federasyon"
- "World Cup" → "Dünya Kupası" (proper name preserved)
- "World Championships" → "Dünya Şampiyonası"

Legal / Regulatory:
- "Regulations" → "Yönetmelik" (DO NOT use "Kurallar")
- "Rules amendments" → "Kural değişiklikleri"
- "Memorandum" → "Memorandum (Duyuru)"
- "Entry into force" → "Yürürlüğe giriş"
- "Eligibility" → "Uygunluk"
- "Licence holder" → "Lisanslı sporcu"
- "Sanction" → "Yaptırım" (DO NOT confuse with "Ceza")
- "Penalty" → "Ceza"
- "Disqualification (DSQ)" → "Diskalifiye (DSQ)"
- "Suspension" → "Men cezası"

Modal / Legal language (VERY IMPORTANT):
- "Shall" → "-mek zorundadır" (NEVER use "-acaktır")
- "Must" → "-melidir"
- "May" → "-ebilir"
- "In accordance with" → "-e uygun olarak"
- "Subject to" → "-e tabi olarak"

Athletes and categories:
- "Rider" → "Sporcu" (NEVER use "Sürücü")
- "Athlete" → "Sporcu"
- "Elite" → "Elit"
- "Junior" → "Gençler"
- "Age category" → "Yaş kategorisi"
- "Under 23" → "23 yaş altı"

Events and competition:
- "Event" → "Yarış / Organizasyon" (choose based on context)
- "Competition" → "Yarışma" (DO NOT confuse with Event)
- "Category" → "Kategori" (DO NOT use "Sınıf")
- "Participation" → "Katılım"

Disciplines and formats:
- "Mountain Bike (MTB)" → "Dağ Bisikleti (MTB)" (explain on first use)
- "Cross-country (XCO)" → "Kros-Kantri (XCO)"
- "Downhill (DHI)" → "İniş (DHI)"
- "Short Track (XCC)" → "Kısa Parkur (XCC)"

Race procedures:
- "Course" → "Parkur" (NEVER use "Pist")
- "Start order" → "Start sıralaması"
- "Qualification" → "Eleme"
- "Heat" → "Seri"

Officials:
- "Commissaire" → "Hakem" (UCI standard)
- "Chief Commissaire" → "Başhakem"
- "Jury" → "Hakem Kurulu"

Output rules:
- Output ONLY the translated Turkish text
- No explanations, no comments, no markdown
"""


In [ ]:
from tqdm import tqdm
import time

In [ ]:
def call_teacher_llm(text):
    prompt = f"""
{SYSTEM_PROMPT}

Text to translate:
{text}
"""
    response = model.generate_content(prompt)
    return response.text.strip()


In [ ]:
INPUT_CSV = "uci_rules_en_chunks.csv"

df = pd.read_csv(INPUT_CSV, encoding="utf-8")

df = df[df["source_en"].notna()].reset_index(drop=True)

df.head()


In [ ]:
TEST_N = 3  # önce 3 satır yeter

test_outputs = []

for i, row in df.head(TEST_N).iterrows():
    uid = row["uid"]
    text = row["source_en"]

    print("="*80)
    print(f"UID: {uid}")
    print("SOURCE (EN):")
    print(text)
    print("\n--- TRANSLATING ---\n")

    tr_text = "" # Initialize tr_text
    for attempt in range(2):  # 1 retry hakkı
        try:
            tr_text = call_teacher_llm(text)
            print("TARGET (TR):")
            print(tr_text)
            break
        except Exception as e:
            if attempt == 1:
                print(f"❌ FAIL | uid={uid} | {e}")
            else:
                print(f"⚠️ Retry for uid={uid} after error: {e}")
                time.sleep(2) # Wait before retrying

    test_outputs.append({
        "uid": uid,
        "article_id": row["article_id"],
        "source_en": text,
        "target_tr": tr_text
    })

    time.sleep(1)

In [ ]:
translated_rows = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    uid = row["uid"]
    text = row["source_en"]

    # Daha uzun bir bekleme süresi ile birden fazla deneme hakkı
    for attempt in range(5):  # 4 retry hakkı
        try:
            tr_text = call_teacher_llm(text)

            translated_rows.append({
                "uid": uid,
                "article_id": row["article_id"],
                "source_en": text,
                "target_tr": tr_text
            })

            time.sleep(1)  # Başarılı çağrılar arasında daha uzun bekleme
            break

        except Exception as e:
            print(f"⚠️ Hata alındı: {e}")
            if attempt < 4: # Son deneme değilse tekrar dene
                # API hatasından 'retry-after' süresini çıkarmaya çalış
                retry_after_match = re.search(r'Please retry in (\d+\.\d+)s', str(e))
                if retry_after_match:
                    wait_time = float(retry_after_match.group(1)) + 1 # Biraz daha ekle
                    print(f"Retrying for uid={uid} after {wait_time:.2f} seconds...")
                    time.sleep(wait_time)
                else:
                    # Varsayılan olarak daha uzun bekleme
                    print(f"Retrying for uid={uid} after 10 seconds (attempt {attempt + 1})...")
                    time.sleep(10)
            else:
                print(f"❌ FAIL | uid={uid} | Maksimum deneme hakkı aşıldı: {e}")



In [ ]:
for i in translated_rows:
  print(translated_rows(i))
  print("\n")